# Exercise 1: Query Expansion and Indexing

The following code is modified from Exercise 1. It is used to construct the vocabulary and vectorize the documents and query.

In [1]:
from nltk.stem import PorterStemmer, WordNetLemmatizer
import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import string
from nltk.corpus import stopwords
import math
from collections import Counter
nltk.download('stopwords')
import numpy as np

stemmer = PorterStemmer()

# Tokenize, stem a document
def tokenize(text):
    text = "".join([ch for ch in text if ch not in string.punctuation])
    tokens = nltk.word_tokenize(text)
    return " ".join([stemmer.stem(word.lower()) for word in tokens])

# Read a list of documents from a file. Each line in a file is a document
#with open("bread.txt") as f:
with open("epfldocs.txt") as f:
    content = f.readlines()
original_documents = [x.strip() for x in content] 
documents = [tokenize(d).split() for d in original_documents]

# create the vocabulary
vocabulary = set([item for sublist in documents for item in sublist])
vocabulary = [word for word in vocabulary if word not in stopwords.words('english')]
vocabulary.sort()

# compute IDF, storing idf values in a dictionary
def idf_values(vocabulary, documents):
    idf = {}
    num_documents = len(documents)
    for i, term in enumerate(vocabulary):
        idf[term] = math.log(num_documents/sum(term in document for document in documents), math.e)
    return idf

# Function to generate the vector for a document (with normalisation)
def vectorize(document, vocabulary, idf):
    vector = [0]*len(vocabulary)
    counts = Counter(document)
    max_count = counts.most_common(1)[0][1]
    for i,term in enumerate(vocabulary):
        vector[i] = idf[term] * counts[term]/max_count
    return vector

# Function to compute cosine similarity
def cosine_similarity(v1,v2):
    sumxx, sumxy, sumyy = 0, 0, 0
    for i in range(len(v1)):
        x = v1[i]; y = v2[i]
        sumxx += x*x
        sumyy += y*y
        sumxy += x*y
    if sumxy == 0:
            result = 0
    else:
            result = sumxy/math.sqrt(sumxx*sumyy)
    return result

def vectorize_query(query, vocabulary, idf):
    q = query.split()
    q = [stemmer.stem(w) for w in q]
    query_vector = vectorize(q, vocabulary, idf)
    return query_vector
    
def search_vec(query, k):
    query_vector = vectorize_query(query, vocabulary, idf)
    scores = [[cosine_similarity(query_vector, document_vectors[d]), d] for d in range(len(documents))]
    scores.sort(key=lambda x: -x[0])
    ans = []
    indices = []
    for i in range(min(k,len(original_documents))):
        ans.append(original_documents[scores[i][1]])
        indices.append(scores[i][1])
    return ans, indices, query_vector

# Compute IDF values and vectors
idf = idf_values(vocabulary, documents)
document_vectors = [vectorize(s, vocabulary, idf) for s in documents]

[nltk_data] Downloading package stopwords to /Users/elmas/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Question 1 - Relevance Feedback

In this exercise we will implement and test Rocchio's method for user relevance feedback.

Let the set of relevant documents to be $D_r $ and the set of non-relevant documents to be $D_{nr}$. Then the modified query  $\vec{q_m}$  according to the Rocchio method is:

\begin{equation}
\vec{q_m} = \alpha \vec{q_0} + \frac{\beta}{|D_r|} \sum_{\vec{d_j} \in D_r} \vec{d_j} - \frac{\gamma}{|D_{nr}|} \sum_{\vec{d_j} \in D_{nr}} \vec{d_j}
\end{equation}
In the Rocchio algorithm negative term weights are ignored. This means, for the negative term weights in $\vec{q_m}$, we set them to be 0.

First, complete the implementation of the Rocchio relevance feedback method, by adding the missing code for the function $expand\_query$.   

Then, compare the result obtained with the new query with the unmodified one.

In [2]:
def expand_query(relevant_doc_vecs, non_relevant_doc_vecs, query_vector, alpha, beta, gamma):
    # Note: relevant_doc_vecs and non_relevant_doc_vecs are list of vectors, vectors are also lists in this case. 
    # We are using (zip(*list)) to columnwise addition. i.e. [[1,2,3], [4,5,6]] iterate over tuples (1,4),(2,5),(3,6)
    # Check here: https://stackoverflow.com/questions/29139350/difference-between-ziplist-and-ziplist
    # You can use numpy if you want to go fancier
    
    num_rel = len(relevant_doc_vecs)
    num_non_rel = len(non_relevant_doc_vecs)
    
    # Compute the first term in the Rocchio equation
    norm_query_vector = query_vector*alpha
    
    # Compute the second term in the Rocchio equation
    norm_sum_relevant = [beta*sum(x)/num_rel for x in zip(*relevant_doc_vecs)]
    
    # Compute the last term in the Rocchio equation
    norm_sum_non_relevant = [-gamma*sum(x)/num_non_rel for x in zip(*non_relevant_doc_vecs)]
    
    # Sum all the terms
    modified_query_vector = [sum(x) for x in zip(norm_sum_relevant, norm_sum_non_relevant, norm_query_vector)]
    
    # Ignore negative elements
    modified_query_vector = [x if x>0 else 0 for x in modified_query_vector]
    return modified_query_vector

In [3]:
ans, result_doc_ids, query_vector = search_vec("computer science", 5)
for i in range(len(ans)):
    print(i,ans[i])

0 Exciting News: "World University Rankings 2016-2017 by subject: computer science" No1 @ETH &amp; @EPFL on No8. Congrats https://t.co/ARSlXZoShQ
1 New computer model shows how proteins are controlled "at a distance" https://t.co/zNjK3bZ6mO  via @EPFL_en #VDtech https://t.co/b9TglXO4KD
2 An interview with Patrick Barth, a new @EPFL professor who combines protein #biophysics with computer modeling https://t.co/iJwBaEbocj
3 New at @epfl_en Life Sciences @epflSV: "From PhD directly to Independent Group Leader" #ELFIR_EPFL:  Early Independence Research Scholars. See https://t.co/evqyqD7FFl, also for computational biology #compbio https://t.co/e3pDCg6NVb Deadline April 1 2018 at https://t.co/mJqcrfIqkb
4 Video of Nicola Marzari from @EPFL_en  on Computational Discovery in the 21st Century during #PASC17 now online: https://t.co/tfCkEvYKtq https://t.co/httPdHcK9W


In [4]:
# list of indices marked as relevant
# suppose first three documents were relevant and the rest were irrelevant.
relevant_indices = [0,1,2]
non_relevant_indices = [i for i in range(3, len(ans))]

relevant_doc_ids = [result_doc_ids[i] for i in relevant_indices]
non_relevant_doc_ids = [result_doc_ids[i] for i in non_relevant_indices]

relevant_doc_vecs = [document_vectors[i] for i in relevant_doc_ids]
non_relevant_doc_vecs = [document_vectors[i] for i in non_relevant_doc_ids]

expanded_query = expand_query(relevant_doc_vecs, non_relevant_doc_vecs, query_vector, 1, 1, 1)

new_query = ' '.join([vocabulary[i] for i, val in enumerate(expanded_query) if val>0])

new_ans , not_important_now, idontcare_anymore = search_vec(new_query, 10)

print('Modified query: ', new_query)
new_ans

Modified query:  20162017 amp barth biophys combin comput congrat control distanc epfl eth excit httpstcoarslxzoshq httpstcob9tglxo4kd httpstcoijwbaebocj httpstcoznjk3bz6mo interview model new news no1 no8 patrick professor protein rank scienc show subject univers vdtech via world


['Exciting News: "World University Rankings 2016-2017 by subject: computer science" No1 @ETH &amp; @EPFL on No8. Congrats https://t.co/ARSlXZoShQ',
 'New computer model shows how proteins are controlled "at a distance" https://t.co/zNjK3bZ6mO  via @EPFL_en #VDtech https://t.co/b9TglXO4KD',
 'An interview with Patrick Barth, a new @EPFL professor who combines protein #biophysics with computer modeling https://t.co/iJwBaEbocj',
 'International ranking: #ETH and #EPFL are top institutes',
 'Interview (in French) de Patrick Aebischer, un "innovation slasher" @EPFL_en https://t.co/BtzhxEAEmN',
 'The proteins that domesticated our genomes https://t.co/npGbUKJhBl  via @EPFL_en #VDtech https://t.co/It0SBqlKQc',
 "New software can model natural light from the occupants' perspective https://t.co/RbMmN3Po5v via @EPFL_en #VDtech https://t.co/50enZtwUHi",
 "New software can model natural light from the occupants' perspective https://t.co/RbMmN3Po5v via @EPFL_en #VDtech https://t.co/lLIAvntc9R",
 'L

# Exercise 2: Link based ranking

### Preliminaries
If you want to normalize a vector to L1-norm or L2-norm, use:

In [5]:
from __future__ import print_function, division
import numpy as np

pr = np.array([1,2,3])
print("L1-norm of {0} is {1}".format(pr, pr / np.linalg.norm(pr,1)))
print("L2-norm of {0} is {1}".format(pr, pr / np.linalg.norm(pr,2)))

L1-norm of [1 2 3] is [0.16666667 0.33333333 0.5       ]
L2-norm of [1 2 3] is [0.26726124 0.53452248 0.80178373]


## Question 1 - Page Rank (Eigen-vector method)
Consider a tiny Web with three pages A, B and C with no inlinks,
and with initial PageRank = 1. Initially, none of the pages link to
any other pages and none link to them. 
Answer the following questions, and calculate the PageRank for
each question.

1. Link page A to page B.
2. Link all pages to each other.
3. Link page A to both B and C, and link pages B and C to A.
4. Use the previous links and add a link from page C to page B.

#### Hints:
We are using the theoretical PageRank computation (without source of rank). See slide "Transition Matrix for Random Walker" in the lecture note. Columns of link matrix are from-vertex, rows of link matrix are to-vertex. We take the eigenvector with the largest eigenvalue.
We only care about final ranking of the probability vector. You can choose the normalization (or not) of your choice).

In [30]:
def create_Rmatrix(L):
#     R = np.multiply(L, 1 / np.sum(L,axis=0)) # Use this matrix multiplication for faster running time if no column is zero
    X = np.sum(L,axis=0)
    n_nodes = L.shape[0]
    R = np.zeros((n_nodes, n_nodes))
    for i in range(L.shape[0]):
        for j in range(L.shape[1]):
            R[i,j] = L[i,j] / X[0,j] if X[0,j] != 0 else 0
            
#     R = np.multiply(L,R)
    return R

In [32]:
"""Some time we might want to compute R outside the function to avoid recomputing large matrix"""
def pagerank_eigen(L, R=None):
#   Construct transition probability matrix from L
    if R is None: R = create_Rmatrix(L)
#     Compute eigen-vectors and eigen-values of R
    eigenvalues, eigenvectors = np.linalg.eig(R)
#     Take the eigen-vector with maximum eigven-value
    p = eigenvectors[:,np.argmax(np.absolute(eigenvalues))]
    return (R,p)


In [33]:
# Test with the question, e.g.
L = np.matrix([
    [0,1,1], 
    [1,0,1], 
    [1,1,0]
])
R,p = pagerank_eigen(L)
print("L={0}\nR={1}\np={2}".format(L,R,p))

L=[[0 1 1]
 [1 0 1]
 [1 1 0]]
R=[[0.  0.5 0.5]
 [0.5 0.  0.5]
 [0.5 0.5 0. ]]
p=[0.57735027 0.57735027 0.57735027]


1.Link page A to page B

Result:
$
L =
  \begin{bmatrix}
	0 & 0 & 0 \\
1 & 0 & 0 \\
0 & 0 & 0 \\
  \end{bmatrix}
 ,
R =
  \begin{bmatrix}
	0 & 0 & 0 \\
1 & 0 & 0 \\
0 & 0 & 0 \\
  \end{bmatrix}
  ,
  \vec{p} =
  \begin{bmatrix}
	0 \\
1 \\
0 \\
  \end{bmatrix}
$

In [34]:
L = np.matrix([
    [0,0,0], 
    [1,0,0], 
    [0,0,0]
])

R,p = pagerank_eigen(L)
print("L={0}\nR={1}\np={2}".format(L,R,p))

L=[[0 0 0]
 [1 0 0]
 [0 0 0]]
R=[[0. 0. 0.]
 [1. 0. 0.]
 [0. 0. 0.]]
p=[0. 1. 0.]


2.Link all pages to each other

Result:
$
L =
  \begin{bmatrix}
	0 & 1 & 1 \\
1 & 0 & 1 \\
1 & 1 & 0 \\
  \end{bmatrix}
 ,
R =
  \begin{bmatrix}
	0 & \frac{1}{2} & \frac{1}{2} \\
\frac{1}{2} & 0 & \frac{1}{2} \\
\frac{1}{2} & \frac{1}{2} & 0 \\
  \end{bmatrix}
  ,
  \vec{p} =
  \begin{bmatrix}
	0.577 \\
0.577 \\
0.577 \\
  \end{bmatrix}
$

In [35]:
L = np.matrix([
    [0,1,1], 
    [1,0,1], 
    [1,1,0]
])

R,p = pagerank_eigen(L)
print("L={0}\nR={1}\np={2}".format(L,R,p))

L=[[0 1 1]
 [1 0 1]
 [1 1 0]]
R=[[0.  0.5 0.5]
 [0.5 0.  0.5]
 [0.5 0.5 0. ]]
p=[0.57735027 0.57735027 0.57735027]


3.Link page A to both B and C, and link pages B and C to A.

Result:
$
L =
  \begin{bmatrix}
	0 & 1 & 1 \\
1 & 0 & 0 \\
1 & 0 & 0 \\
  \end{bmatrix}
 ,
R =
  \begin{bmatrix}
	0 & 1 & 1 \\
\frac{1}{2} & 0 & 0 \\
\frac{1}{2} & 0 & 0 \\
  \end{bmatrix}
  ,
  \vec{p} =
  \begin{bmatrix}
	0.816 \\
0.408 \\
0.408 \\
  \end{bmatrix}
$

In [36]:
L = np.matrix([
    [0,1,1], 
    [1,0,0], 
    [1,0,0]
])

R,p = pagerank_eigen(L)
print("L={0}\nR={1}\np={2}".format(L,R,p))

L=[[0 1 1]
 [1 0 0]
 [1 0 0]]
R=[[0.  1.  1. ]
 [0.5 0.  0. ]
 [0.5 0.  0. ]]
p=[0.81649658 0.40824829 0.40824829]


4.Use the previous links and add a link from page C to page B

Result:
$
L =
  \begin{bmatrix}
	0 & 1 & 1 \\
1 & 0 & 1 \\
1 & 0 & 0 \\
  \end{bmatrix}
 ,
R =
  \begin{bmatrix}
	0 & 1 & \frac{1}{2} \\
\frac{1}{2} & 0 & \frac{1}{2} \\
\frac{1}{2} & 0 & 0 \\
  \end{bmatrix}
  ,
  \vec{p} =
  \begin{bmatrix}
	0.743 \\
0.557 \\
0.371 \\
  \end{bmatrix}
$

In [37]:
L = np.matrix([
    [0,1,1], 
    [1,0,1], 
    [1,0,0]
])

R,p = pagerank_eigen(L)
print("L={0}\nR={1}\np={2}".format(L,R,p))

L=[[0 1 1]
 [1 0 1]
 [1 0 0]]
R=[[0.  1.  0.5]
 [0.5 0.  0.5]
 [0.5 0.  0. ]]
p=[-0.74278135+0.j -0.55708601+0.j -0.37139068+0.j]


# Question 2 - Page Rank (Iterative method)

The eigen-vector method has some numerical issues (when computing eigen-vector) and not scalable with large datasets.

We will apply the iterative method in the slide "Practical Computation of PageRank" of the lecture.

Dataset for practice: https://snap.stanford.edu/data/ca-GrQc.html. It is available within the same folder of this github.

In [38]:
def pagerank_iterative(L, R=None):
    if R is None: #We might want to compute R outside this function to avoid recomputing large matrix
        R = np.multiply(L, 1 / np.sum(L,axis=0))
        
    N = R.shape[0]
    e = np.ones(shape=(N,1))
    q = 0.9

    p = e
    delta = 1
    epsilon = 0.001
    i = 0
    while delta > epsilon:
        p_prev = p
        p = np.matmul(q * R, p_prev)
        p = p + (1-q) / N * e
        delta = np.linalg.norm(p-p_prev,1)
        i += 1

    print("Converged after {0} iterations".format(i))
    return R,p

#### Construct link matrix from dataset

In [39]:
n_nodes = 0
nodes_idx = dict() #Since the nodeIDs are not from 0 to N we need to build an index of nodes
nodes = [] #We also want to store nodeIDs to return the result of ranking vector

# Read the nodes
with open("ca-GrQc.txt") as f:
    for line in f:
        if '#' not in line:
            source = int(line.split()[0])
            target = int(line.split()[1])
            if source not in nodes_idx.keys():
                nodes_idx[source] = n_nodes
                nodes.append(source)
                n_nodes += 1
            if target not in nodes_idx.keys():
                nodes_idx[target] = n_nodes
                nodes.append(target)
                n_nodes += 1
print(n_nodes)
print(nodes[:3])

5242
[3466, 937, 5233]


In [40]:
L = np.zeros((n_nodes, n_nodes))
# Read the edges
with open("ca-GrQc.txt") as f:
    for line in f:
        if "#" not in line:
            source = int(line.split()[0])
            target = int(line.split()[1])
            L[nodes_idx[target], nodes_idx[source]] = 1 #Columns of link matrix are from-vertices
print(L)

[[0. 1. 1. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 1. 1.]
 [0. 0. 0. ... 1. 0. 1.]
 [0. 0. 0. ... 1. 1. 0.]]


#### Compute transition probability matrix

In [41]:
# Here I use matrix multiplication from numpy for faster running time
R = np.multiply(L, 1 / np.sum(L,axis=0))
# R = create_Rmatrix(L) # This is much slower
print(R)



[[0.    0.2   0.5   ... 0.    0.    0.   ]
 [0.125 0.    0.    ... 0.    0.    0.   ]
 [0.125 0.    0.    ... 0.    0.    0.   ]
 ...
 [0.    0.    0.    ... 0.    0.5   0.5  ]
 [0.    0.    0.    ... 0.5   0.    0.5  ]
 [0.    0.    0.    ... 0.5   0.5   0.   ]]


#### You will see that eigen-vector method is slow and has some numerical issues


In [42]:
import time
start_time = time.time()
R,p = pagerank_eigen(L, R)
print("--- %s seconds ---" % (time.time() - start_time))
print("Ranking vector: p={0}".format(p))
# print(eigenvectors)

--- 88.81494784355164 seconds ---
Ranking vector: p=[0.01151345+0.j 0.00719591+0.j 0.00287836+0.j ... 0.        +0.j
 0.        +0.j 0.        +0.j]


#### Iterative method

In [43]:
import time
start_time = time.time()
R, p = pagerank_iterative(L,R)
print("--- %s seconds ---" % (time.time() - start_time))
print("Ranking vector: p={0}".format(p[:,0]))

Converged after 128 iterations
--- 16.894638061523438 seconds ---
Ranking vector: p=[2.91315639e-04 1.88382754e-04 8.39741651e-05 ... 1.92156702e-04
 1.92156702e-04 1.92156702e-04]


#### Extract top-k nodes from the ranking vector

In [44]:
arr = np.array(p[:,0])
k = 3
k_idx = arr.argsort()[-k:][::-1]
print("Top-{0} nodes: {1}".format(k, np.array(nodes)[k_idx]))
print("Their scores: {0}".format(arr[k_idx]))

Top-3 nodes: [14265 13801 13929]
Their scores: [0.00144951 0.00141553 0.00138011]


## Question 3 - Ranking Methodology (Hard)

1. Give a directed graph, as small as possible, satisfying all the properties mentioned below:

    1. There exists a path from node i to node j for all nodes i,j in the directed
graph. Recall, with this property the jump to an arbitrary node in PageRank
is not required, so that you can set q = 1 (refer lecture slides).

    2. HITS authority ranking and PageRank ranking of the graph nodes are different.

2. Give intuition/methodology on how you constructed such a directed graph with
the properties described in (a).

3. Are there specific graph structures with arbitrarily large instances where PageRank
ranking and HITS authority ranking are the same?

Solutions in pdf..

## Question 4 - Hub and Authority

### a)

Let the adjacency matrix for a graph of four vertices ($n_1$ to $n_4$) be
as follows:

$
A =
  \begin{bmatrix}
	0 & 1 & 1 & 1  \\
	0 & 0 & 1 & 1 \\
	1 & 0 & 0 & 1 \\
	0 & 0 & 0 & 1 \\
  \end{bmatrix}
$

Calculate the authority and hub scores for this graph using the
HITS algorithm with k = 6, and identify the best authority and
hub nodes.

### b)
Apply the HITS algorithm to the dataset: https://snap.stanford.edu/data/ca-GrQc.html

**Hint:** We follow the slide "HITS algorithm" in the lecture. **Denote $x$ as authority vector and $y$ as hub vector**. You can use matrix multiplication for the update steps in the slide "Convergence of HITS". Note that rows of adjacency matrix is from-vertex and columns of adjacency matrix is to-vertex.

In [45]:
def hits_iterative(A, k = 10):
    N = A.shape[0]
    x0, y0 = 1 / (N*N) * np.ones(N), 1 / (N*N) * np.ones(N) 
    xprev, yprev = x0, y0
    delta1 = delta2 = 1
    epsilon = 0.001 # We can strictly check for convergence rate of HITS algorithm
    l = 0
    while l < k and delta1 > epsilon and delta2 > epsilon:
        y = np.matmul(A, xprev)
        x = np.matmul(np.transpose(A), y) 
        x = x / np.linalg.norm(x,2)
        y = y / np.linalg.norm(y,2)
        delta1 = np.linalg.norm(x-xprev,1)
        delta2 = np.linalg.norm(y-yprev,1)
        xprev = x
        yprev = y
        l += 1
    
    print("Ran a total of {0} iterations with the convergence rate delta1, delta2={1},{2}".format(l, delta1, delta2))
    return xprev, yprev

#### a)

In [46]:
A=np.array([
    [0, 1, 1, 1], 
    [0, 0, 1, 1], 
    [1, 0, 0, 1],
    [0, 0, 0, 1],
])

x, y = hits_iterative(A, 100)
print("Result using iterative method:\n Authoriy vector x={0}\n Hub vector y={1}".format(x, y))

Ran a total of 9 iterations with the convergence rate delta1, delta2=0.0007655728389590333,0.0016260291592009035
Result using iterative method:
 Authoriy vector x=[0.16878931 0.27236456 0.49773665 0.80596894]
 Hub vector y=[0.65546933 0.54214151 0.40516824 0.33508393]


**Details:**
+ Initialization: 
  
  $x_0 = \frac{1}{4^2}(1,1,1,1) = ( 0.0625,  0.0625,  0.0625,  0.0625)$
  
  $y_0 = \frac{1}{4^2}(1,1,1,1) = ( 0.0625,  0.0625,  0.0625,  0.0625)$
  
+ $k=1$:
  
  $x_1 = \frac{A^t y_0}{||A^t y_0||} = (0.21320072,  0.21320072,  0.42640143,  0.85280287)$
  
  $y_1 = \frac{A x_0}{|| A x_0 ||} = (0.70710678,  0.47140452,  0.47140452,  0.23570226)$
  
+ ...:
  
+ $k=6$:
  
   $x_6 = \frac{A^t y_5}{||A^t y_5||} = (0.16887796,  0.27257494,  0.49774555, 0.80587375)$
  
  $y_6 = \frac{A x_5}{||A x_5||} = (0.65357971,  0.54153747,  0.40815386,  0.33612671)$
  

**Conclusion:**
+ Best authority node: $n_4$. Best hub node: $n_1$.
 
**Check with the theoretical result (convergence condition):**
  
+ $x^*$ is the principal eigenvector (i.e. with largest eigenvalue) of $A^t A$: $(0.16845787,  0.27257056,  0.49801119,  0.80579904)$
  
+ $y^*$ is the principal eigenvector (i.e. with largest eigenvalue) of $A A^t$: $(0.65549599,  0.54215478,  0.4051188,   0.33507008)$

### b)
We reuse the link matrix $L$ to compute the adjacency matrix $A$ of the dataset

In [47]:
A = np.transpose(L)
print(A)

[[0. 1. 1. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 1. 1.]
 [0. 0. 0. ... 1. 0. 1.]
 [0. 0. 0. ... 1. 1. 0.]]


Run HITS algorithm:

In [48]:
x, y = hits_iterative(A, 100)
print("Result using iterative method:\n Authoriy vector x={0}\n Hub vector y={1}".format(x, y))

Ran a total of 39 iterations with the convergence rate delta1, delta2=0.0009846048386951208,0.000984604838696764
Result using iterative method:
 Authoriy vector x=[3.48948810e-05 1.45965389e-05 6.68131703e-06 ... 1.28245551e-54
 1.28245551e-54 1.28245551e-54]
 Hub vector y=[3.48948810e-05 1.45965389e-05 6.68131703e-06 ... 1.28245551e-54
 1.28245551e-54 1.28245551e-54]


**You can see that the two authority vector and hub vector are the same. So the network must be an undirected graph**

**Interpret the result:**

In [49]:
k = 3

arr = np.array(x)
k_idx = arr.argsort()[-k:][::-1]
print("Top-{0} authorities: {1}".format(k, np.array(nodes)[k_idx]))
print("Their scores: {0}".format(arr[k_idx]))

arr = np.array(y)
k_idx = arr.argsort()[-k:][::-1]
print("Top-{0} hubs: {1}".format(k, np.array(nodes)[k_idx]))
print("Their scores: {0}".format(arr[k_idx]))

Top-3 authorities: [21012  2741 12365]
Their scores: [0.15556238 0.15357488 0.15307255]
Top-3 hubs: [21012  2741 12365]
Their scores: [0.15556238 0.15357488 0.15307255]


**We can also use linear algebra property of HITS to compute the result (slide "Convergence of HITS"):**.
  
+ $x^*$ is the principal eigenvector (i.e. with largest eigenvalue) of $A^t A$
+ $y^*$ is the principal eigenvector (i.e. with largest eigenvalue) of $A A^t$

However, the computation will be much slower

In [50]:
xstar_ev, xstar = np.linalg.eig(np.matmul(np.transpose(A),A))
ystar_ev, ystar = np.linalg.eig(np.matmul(A,np.transpose(A)))
xstar, ystar = xstar[:,np.argmax(np.absolute(xstar_ev))], ystar[:,np.argmax(np.absolute(ystar_ev))]
# ystar = -xstar if all(xstar<0) else xstar
# ystar = -ystar if all(ystar<0) else ystar
print("Result using linear algebra:\n Authoriy vector x={0}\n Hub vector y={1}".format(xstar, ystar))

Result using linear algebra:
 Authoriy vector x=[3.48948725e-05+0.j 1.45965100e-05+0.j 6.68130767e-06+0.j ...
 0.00000000e+00+0.j 0.00000000e+00+0.j 0.00000000e+00+0.j]
 Hub vector y=[3.48948725e-05+0.j 1.45965100e-05+0.j 6.68130767e-06+0.j ...
 0.00000000e+00+0.j 0.00000000e+00+0.j 0.00000000e+00+0.j]
